In [1]:
import os
import os.path as osp
import json
import imageio
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
import plotly.graph_objects as go
import matplotlib.animation as animation

from radar_dataclass import *
from natsort import natsorted
from radar_data_preprocess import RadarData
from typing import List, Dict
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

warnings.filterwarnings('ignore')

In [ ]:
RADAR_DATA_DIR = '../radar/json/'

data = RadarData(root_path=RADAR_DATA_DIR, duration_frames=0)

In [3]:
frame_dict: Dict[int, Radar] = data.frame_dict
frame_info_list: List[frameHeader] = [frame_dict[keys].frameInfo for keys in frame_dict]

dets_list: List[detPointsStruct] = [frame_dict[keys].dets for keys in frame_dict]

In [ ]:
frame_time_list = list()

for frame_info in frame_info_list:
    frame_time_list.append(frame_info.add_frameTime_ms)

plt.plot(range(len(frame_time_list)), frame_time_list)
plt.grid(linestyle='--')

In [ ]:
frame_diff_time_list = list()
frame_ref_time = frame_info_list[0].add_frameTime_ms

for frame_info in frame_info_list[1:]:
    frame_diff_time_list.append(frame_info.add_frameTime_ms - frame_ref_time)
    frame_ref_time = frame_info.add_frameTime_ms

plt.plot(range(len(frame_diff_time_list)), frame_diff_time_list)
plt.grid(linestyle='--')

In [ ]:
sns.distplot(frame_diff_time_list, bins=100)

In [ ]:
len(dets_list[2].dets)

In [ ]:
det_nums_list = list()

# for frame_info in frame_info_list:
#     det_nums_list.append(frame_info.)

for det_frame in dets_list:
    det_nums_list.append(len(det_frame.dets))

plt.figure(figsize=(100,3))
plt.plot(range(len(det_nums_list)), det_nums_list)
plt.grid(linestyle='--')

In [ ]:
plt.scatter(frame_diff_time_list, det_nums_list[1:])

In [ ]:
trk_nums_list = list()

for frame_info in frame_info_list:
    trk_nums_list.append(frame_info.numTracker)

plt.figure(figsize=(100,3))
plt.plot(range(len(trk_nums_list)), trk_nums_list)
plt.grid(linestyle='--')

In [ ]:
plt.scatter(frame_diff_time_list, trk_nums_list[1:])

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=frame_diff_time_list,
    y=trk_nums_list[1:],
    z=det_nums_list[1:],
    mode='markers',  # 'lines', 'markers', 'lines+markers'
    marker=dict(
        size=1,
        color=det_nums_list[1:],    
        colorscale='Viridis',
        opacity=1
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
)

In [ ]:
# frame_idx = 1


# import matplotlib.animation as animation

# fig, ax = plt.subplots()
# # cmap = plt.get_cmap('viridis')
# # norm = plt.Normalize(vmin=min([item.doppler for det_frame in dets_list for item in det_frame.dets]), vmax=max([item.doppler for det_frame in dets_list for item in det_frame.dets]))

# def update(frame_idx):
#     ax.clear()
#     single_frame_dets_array = np.array([np.array([item.add_pos_x, item.add_pos_y, item.doppler]) for item in dets_list[frame_idx].dets])

#     ax.scatter(single_frame_dets_array[:, 1], single_frame_dets_array[:, 0], s=0.8, )
#     ax.set_title(f'Frame {frame_idx}')
#     ax.set_xlim(-100, 100)
#     ax.set_ylim(-10, 500)
#     ax.grid(True)

# ani = animation.FuncAnimation(fig, update, frames=len(dets_list), repeat=False)
# ani.save('radar_animation.gif', writer='imagemagick')


fig, ax = plt.subplots()

# 计算所有帧中的doppler值的全局最小值和最大值
doppler_all_frames = []

# 收集所有帧的doppler值
for frame_idx in range(len(dets_list)):
    single_frame_dets_array = np.array([np.array([item.add_pos_x, item.add_pos_y, item.doppler]) for item in dets_list[frame_idx].dets])
    doppler_all_frames.extend(np.abs(single_frame_dets_array[:, 2]))  # 添加doppler值到doppler_all_frames

# 计算doppler值的全局最小值和最大值
doppler_min = min(doppler_all_frames)
doppler_max = max(doppler_all_frames)

# 创建一个颜色条并将其添加到图中（只做一次）
cmap = 'viridis'
scatter = ax.scatter([], [], c=[], s=10, cmap=cmap, vmin=doppler_min, vmax=doppler_max)  # 设置vmin和vmax
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Doppler')

def update(frame_idx):
    ax.clear()
    
    # 提取当前帧的检测数据
    single_frame_dets_array = np.array([np.array([item.add_pos_x, item.add_pos_y, item.doppler]) for item in dets_list[frame_idx].dets])
    
    # 使用doppler值直接设置颜色
    doppler_vals = single_frame_dets_array[:, 2]  # 获取doppler值
    
    # 绘制散点图，使用doppler值进行颜色映射
    scatter = ax.scatter(single_frame_dets_array[:, 1], single_frame_dets_array[:, 0], c=doppler_vals, s=0.8, cmap=cmap, vmin=doppler_min, vmax=doppler_max)
    
    ax.set_title(f'Frame {frame_idx}')
    ax.set_xlim(-100, 100)
    ax.set_ylim(-10, 500)
    ax.grid(True)

ani = animation.FuncAnimation(fig, update, frames=len(dets_list), repeat=False)
ani.save('radar_animation.gif', writer='imagemagick')

In [15]:
# plt.imshow(np.array([item['data'] for item in rd_data.rangeDoppler]).reshape(512, -1).T)